# CNN 图像分类

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
print("PyTorch Version: ",torch.__version__)

PyTorch Version:  1.2.0


定义简单的ConvNet神经网络

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        '''
         1：input channel
         20：output channel
         5： kernel size (单个数字时表示长宽相等的filter ，两个数字时分别表示长和宽)
         1： stride （步长）
        '''
        self.conv1 = nn.Conv2d(1,20,5,1)
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500,10)
    
    def forward(self,x):
         # x:1 * 28 * 28 -> 其中 1 指的是 channel 因为黑白的所以是 1 
        x = F.relu(self.conv1(x)) # 20 * （28+1-5） * （28+1-5）--》 20 * 24 * 24
        '''
        max_pool2d()
        x:input
        2:kernel size
        2:stride
        '''
        x = F.max_pool2d(x,2,2) # 使用的pooling size 是 2 所以 --》20 * 12 * 12
        x = F.relu(self.conv2(x))# 50 * 12 * 12
        x = F.max_pool2d(x,2,2) # 50 * 4 * 4
        x = x.view(-1,4*4*50) # 将数据整合成二维的
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        y = F.log_softmax(x,dim =1)
        return y

In [8]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    for idx,(data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        pred = model(data) # batch_size * 10
        ''' 
        torch.nn.functional.nll_loss(input, 
                                     target,
                                     weight=None,
                                     size_average=None,
                                     ignore_index=-100,
                                     reduce=None,
                                     reduction='mean')
        '''
        loss = F.nll_loss(pred,target)
        
        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx %100 ==0:
            print("Train Epoch:{},iteration:{},Loss:{}".format(epoch,idx,loss.item()))

In [24]:
def test(model, device, test_loader):
    model.eval()
    
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx,(data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            ''' loss 的 reduction 默认是 mean ：求解loss的平均值'''
            total_loss += F.nll_loss(output,target,reduction="sum")
            pred = output.argmax(dim=1) #获取第一个维度的最大值的idx
            correct += pred.eq(target.view_as(pred)).sum().item()


        total_loss /= len(test_loader.dataset)
        acc = correct /len(test_loader.dataset) * 100.
        print("Test loss:{},Accracy:{}".format(total_loss, acc))

加载数据集

In [25]:
mnist_data = datasets.MNIST("./mnist_data",
                            train = True,
                            download = True,
                            transform = transforms.Compose([
                                transforms.ToTensor(),
                            ])) # 如果路径下没有数据集会自动下载

In [26]:
# mnist data set 是 一个 dataset 包含多个方法使用 dir（mnist_data） 查看 mnist_data 的方法
#dir(mnist_data)
#mnist_data[2][0] #可以查看数据集的图片
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data",
                            train = True,
                            download = True,
                            transform = transforms.Compose([
                                transforms.ToTensor(),
                            ])),
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 1,
    pin_memory = True
)
test_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data",
                            train = False,
                            download = True,
                            transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,),(0.3081,))
                            ])),
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 1,
    pin_memory = True
)

超参数

In [27]:
lr = 0.01
momentum = 0.5

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = momentum)
num_epochs = 2

In [28]:
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
    
torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch:0,iteration:0,Loss:2.300945281982422
Train Epoch:0,iteration:100,Loss:2.1642022132873535
Train Epoch:0,iteration:200,Loss:0.8094296455383301
Train Epoch:0,iteration:300,Loss:0.6092169880867004
Train Epoch:0,iteration:400,Loss:0.43711647391319275
Train Epoch:0,iteration:500,Loss:0.4811740815639496
Train Epoch:0,iteration:600,Loss:0.31888359785079956
Train Epoch:0,iteration:700,Loss:0.1649811863899231
Train Epoch:0,iteration:800,Loss:0.29303786158561707
Train Epoch:0,iteration:900,Loss:0.4696680009365082
Train Epoch:0,iteration:1000,Loss:0.19931136071681976
Train Epoch:0,iteration:1100,Loss:0.15847750008106232
Train Epoch:0,iteration:1200,Loss:0.10180233418941498
Train Epoch:0,iteration:1300,Loss:0.21272359788417816
Train Epoch:0,iteration:1400,Loss:0.1466711461544037
Train Epoch:0,iteration:1500,Loss:0.14425426721572876
Train Epoch:0,iteration:1600,Loss:0.1450444906949997
Train Epoch:0,iteration:1700,Loss:0.08176448941230774
Train Epoch:0,iteration:1800,Loss:0.12413038313388

LeNet -> AlexNet -> ResNet -> DanseNet